Within this notebook we scrape the world bank blog post webpages.

## NOTE
Remember to have your `VPN` active.

In [ ]:
import time, random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common import exceptions as e
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidArgumentException, StaleElementReferenceException

In [ ]:
from tqdm import tqdm
from IPython.display import Markdown, display

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.implicitly_wait(10)

In [ ]:
url = 'https://blogs.worldbank.org/search?keyword=&f%5B0%5D=countries%3A143&f%5B1%5D=language%3Aen'
driver.get(url)

### Blog Post URLs

In [ ]:
all_urls = []

while True:
    time.sleep(random.uniform(3, 4))
    blog_posts = driver.find_elements(By.XPATH, "//div[@class='blog_teaser']")
    try:
        url = [blog.find_element(By.TAG_NAME, 'h2').find_element(By.TAG_NAME, 'a').get_attribute('href') for blog in blog_posts]
    except StaleElementReferenceException:
        driver.refresh()
        continue
    next_page = driver.find_element(By.XPATH, "//a[@title='Go to next page']")
    next_page.click()
    
    all_urls.append(url)

In [ ]:
driver.quit()

### Postprocessing

In [ ]:
def flatten_list(nested_list):
    flattened_list = []
    for element in nested_list:
        if isinstance(element, list):
            flattened_list.extend(flatten_list(element))
        else:
            flattened_list.append(element)
    return flattened_list

In [ ]:
all_urls_flattened = flatten_list(all_urls)
print(len(all_urls_flattened))
all_urls_flattened[:5]

### Save and Read

In [ ]:
with open("data/world_bank_document_urls.txt", "w", encoding='utf-8') as f:
    for url in all_urls_flattened:
        f.write(url)

## Articles

In [26]:
from async_get import acreate_coroutines

from bs4 import BeautifulSoup
from IPython.display import display, Markdown, HTML

### Loading

In [2]:
with open("data/world_bank_document_urls.txt", encoding='utf-8') as f:
    article_urls = f.readlines()
    article_urls = [url.replace('\n', '') for url in article_urls]
len(article_urls)

1120

In [3]:
article_urls = list(set(article_urls))          # remove duplicates

### HTTP Get

In [ ]:
articles = await acreate_coroutines(article_urls)

### Extract text and useful metadata

In [56]:
article_texts = []
for article in tqdm(articles):
    soup = BeautifulSoup(article, 'html.parser')
    
    try:
        title = soup.find('h1', class_="blog_teaser__title").text
        authors = soup.find('div', class_="blog_teaser__link_container").text
        body = soup.find('div', class_="cmp-text").text

    except AttributeError:
        title = None
        authors = None
        body = soup.find('body').text


    article_text = f"""
        Title:      {title}
        
        Authors & Date Published : {authors}    

        Article Body:   {body}
    """
    article_texts.append(article_text)

### Save Article Text

In [57]:
with open("data/world_bank_articles.txt", "w", encoding='utf-8') as f:
    for article_text in article_texts:
        f.write("\n\n" + article_text)
        f.write("\n\n" + "-" * 150)

## Legacy
This is deprecated as it is costly both in terms of time and compute resources. The use of asynchronous get requests supercedes it. In fact, it leaves it in the dust.

In [ ]:
# driver = webdriver.Chrome()
# def get_article_text():
#     driver.switch_to.default_content()
#     sections = driver.find_elements(By.XPATH, "//section[@class='mainsection']")
    
#     return sections[0].text
# articles = []
# skipped = []
# for article_no, article_url in enumerate(tqdm(article_urls)):
#     try:
#         driver.get(article_url)

#         article = driver.find_element(By.TAG_NAME, 'body').text
#         # article = get_article_text()
#         articles.append(article)
        
#     except Exception as e:
#         skipped.append(article_no)
#         print(f"The following occurred on article number {article_no}: {e} \nLoading next article...")
#         continue

#     time.sleep(random.uniform(1.5, 2.5))